### Assignment 15: Solving the Tower of Hanoi Problem Using Adversarial Search
Transform the traditional Tower of Hanoi puzzle into a competitive two-player game. Implement adversarial search strategies using the Minimax algorithm to solve the problem. Enhance efficiency by incorporating alpha-beta pruning.

Problem Setup:

The Tower of Hanoi involves three rods and a set of N disks of decreasing size
stacked on the first rod. The goal is to move all the disks to the last rod following these rules:
1. Only one disk can be moved at a time.
2. A disk can only be placed on top of a larger disk or an empty rod.
3. Moves alternate between two players (AI and Human).

The player who moves the largest disk to the target rod wins. If no valid moves are left and the game hasn't ended, it is considered a draw.

Game Design:
1. Represent the rods and disks using lists (e.g., rods = [[3, 2, 1], [], []]
for N=3, N=3, N=3).
2. Create a function to validate moves and update the rods.

Adversarial Search:
1. Implement the Minimax algorithm to evaluate all possible moves for
both players.
2. Define a utility function:
  * Assign positive scores for moves favorable to the AI (e.g., moving larger disks closer to the target rod).
  * Assign negative scores for moves favorable to the opponent.

Alpha-Beta Pruning:
1. Integrate alpha-beta pruning to optimize the search process by pruning unnecessary branches.

Player Interaction:
1. Alternate turns between the AI and the human player.
2. Display the state of the rods after each move.

End Condition:
1. The game ends when:
  * All disks are moved to the target rod.
  * No valid moves are left.
2. The winner is determined based on the size of the largest disk moved to the target rod by each player.

In [ ]:
import math
import copy

N = 3

# State representation: dictionary with keys 'rods' and 'turn'
def initial_state(N):
    return {
        'rods': [list(range(N, 0, -1)), [], []],
        'turn': 'Human',  # Let Human go first; change to 'AI' to start with the AI
        'last_mover': None  # To record who moved a disk (important for win condition)
    }

def print_state(state):
    rods = state['rods']
    print("Rod 0:", rods[0])
    print("Rod 1:", rods[1])
    print("Rod 2:", rods[2])
    print("Current turn:", state['turn'])
    print("-" * 30)

# Check if a move is valid:
# A move is a tuple (source, target).
def is_valid_move(state, move):
    src, tgt = move
    rods = state['rods']
    if src < 0 or src > 2 or tgt < 0 or tgt > 2 or src == tgt:
        return False
    if len(rods[src]) == 0:
        return False
    # Only allow placing on empty rod or on a larger disk.
    if len(rods[tgt]) > 0 and rods[src][-1] > rods[tgt][-1]:
        return False
    return True

# Generate all valid moves from the current state.
def get_valid_moves(state):
    moves = []
    for src in range(3):
        for tgt in range(3):
            if is_valid_move(state, (src, tgt)):
                moves.append((src, tgt))
    return moves

# Make a move and return a new state.
# Also checks if the move wins the game (i.e. moving the largest disk to rod 2).
def make_move(state, move):
    new_state = copy.deepcopy(state)
    src, tgt = move
    disk = new_state['rods'][src].pop()
    new_state['rods'][tgt].append(disk)
    new_state['last_mover'] = state['turn']
    new_state['turn'] = 'AI' if state['turn'] == 'Human' else 'Human'
    return new_state

# Terminal test: game is over if either
# 1. The largest disk is on rod 2.
# 2. No valid moves remain.
def is_terminal(state):
    rods = state['rods']
    # Winning condition: largest disk N is on rod2.
    if N in rods[2]:
        return True
    if len(get_valid_moves(state)) == 0:
        return True
    return False

# Evaluation function:
# - If terminal: return a large positive value if AI wins, large negative if Human wins.
# - Otherwise, use a heuristic: score based on which rod the largest disk is on.
def evaluate(state):
    rods = state['rods']
    if N in rods[2]:
        if state['last_mover'] == 'AI':
            return 1000
        else:
            return -1000
    # Heuristic: assign a value based on the position of the largest disk.
    # (Rod 0: score 0, Rod 1: score 50, Rod 2: score 100)
    score_map = {0: 0, 1: 50, 2: 100}
    for rod_index in range(3):
        if N in rods[rod_index]:
            heuristic_score = score_map[rod_index]
            break
    return heuristic_score

# Minimax algorithm with alpha-beta pruning.
def minimax(state, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or is_terminal(state):
        return evaluate(state), None

    valid_moves = get_valid_moves(state)
    best_move = None

    if maximizingPlayer:
        maxEval = -math.inf
        for move in valid_moves:
            child_state = make_move(state, move)
            eval, _ = minimax(child_state, depth - 1, alpha, beta, False)
            if eval > maxEval:
                maxEval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval, best_move
    else:
        minEval = math.inf
        for move in valid_moves:
            child_state = make_move(state, move)
            eval, _ = minimax(child_state, depth - 1, alpha, beta, True)
            if eval < minEval:
                minEval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval, best_move

def play_game():
    state = initial_state(N)
    print("Initial State:")
    print_state(state)

    while not is_terminal(state):
        if state['turn'] == 'Human':
            valid_moves = get_valid_moves(state)
            if not valid_moves:
                print("No valid moves for Human. Game is a draw!")
                break
            print("Valid moves (source, target):", valid_moves)
            try:
                user_input = input("Enter your move as 'src tgt': ")
                src, tgt = map(int, user_input.split())
                move = (src, tgt)
                if move not in valid_moves:
                    print("Invalid move! Try again.")
                    continue
            except Exception as e:
                print("Error in input. Please enter two numbers separated by a space.")
                continue
            state = make_move(state, move)
        else:
            valid_moves = get_valid_moves(state)
            if not valid_moves:
                print("No valid moves for AI. Game is a draw!")
                break
            _, ai_move = minimax(state, depth=10, alpha=-math.inf, beta=math.inf, maximizingPlayer=True)
            print("AI chooses move:", ai_move)
            state = make_move(state, ai_move)

        print_state(state)

    if N in state['rods'][2]:
        print("Game Over! Winner is:", state['last_mover'])
    else:
        print("Game ended in a draw!")

play_game()

Initial State:
Rod 0: [3, 2, 1]
Rod 1: []
Rod 2: []
Current turn: Human
------------------------------
Valid moves (source, target): [(0, 1), (0, 2)]
Enter your move as 'src tgt': 0 2
Rod 0: [3, 2]
Rod 1: []
Rod 2: [1]
Current turn: AI
------------------------------
AI chooses move: (0, 1)
Rod 0: [3]
Rod 1: [2]
Rod 2: [1]
Current turn: Human
------------------------------
Valid moves (source, target): [(1, 0), (2, 0), (2, 1)]
Enter your move as 'src tgt': 2 0
Rod 0: [3, 1]
Rod 1: [2]
Rod 2: []
Current turn: AI
------------------------------
AI chooses move: (0, 2)
Rod 0: [3]
Rod 1: [2]
Rod 2: [1]
Current turn: Human
------------------------------
Valid moves (source, target): [(1, 0), (2, 0), (2, 1)]
Enter your move as 'src tgt': 2 1
Rod 0: [3]
Rod 1: [2, 1]
Rod 2: []
Current turn: AI
------------------------------
AI chooses move: (0, 2)
Rod 0: []
Rod 1: [2, 1]
Rod 2: [3]
Current turn: Human
------------------------------
Game Over! Winner is: AI
